In [1]:
import re
import csv

# Abrir archivo y leer contenido
with open("honey-biosamples.txt", "r") as f:
    contenido = f.read()

# Separar las entradas por número (1:, 2:, ...)
entradas = re.split(r"\n\d+:\s", contenido)[1:]  # omitir primera entrada vacía
nombres = re.findall(r"\n\d+:\s(.+)", contenido)  # capturar los nombres de las muestras

# print(entradas)
# print(nombres)

# Inicializar lista de diccionarios
datos = []

for nombre, entrada in zip(nombres, entradas):
    biosample = re.search(r"BioSample:\s*(SAMN\d+)", entrada)
    host = re.search(r'/host="([^"]+)"', entrada)
    accession = re.search(r"Accession:\s*(SAMN\d+)", entrada)
    # geographic_location =  re.search(r"Accession:\s*(SAMN\d+)", entrada)
    
    datos.append({
        "Nombre de muestra": nombre,
        "BioSample": biosample.group(1) if biosample else "",
        "Host": host.group(1) if host else "",
        "Accession": accession.group(1) if accession else ""
    })

# Guardar como CSV
with open("honey-hosts.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["Nombre de muestra", "BioSample", "Host", "Accession"])
    writer.writeheader()
    writer.writerows(datos)



In [2]:
import re
import csv

# Abrir archivo y leer contenido
with open("honey-biosamples.txt", "r") as f:
    contenido = f.read()

# Separar las entradas por número (1:, 2:, ...)
entradas = re.split(r"\n\d+:\s", contenido)[1:]
nombres = re.findall(r"\n\d+:\s(.+)", contenido)

# Inicializar lista de diccionarios
datos = []

for nombre, entrada in zip(nombres, entradas):
    biosample = re.search(r"BioSample:\s*(SAMN\d+)", entrada)
    host = re.search(r'/host="([^"]+)"', entrada)
    accession = re.search(r"Accession:\s*(SAMN\d+)", entrada)
    geo_loc = re.search(r'/geographic location="([^"]+)"', entrada)
    lat_lon = re.search(r'/latitude and longitude="([^"]+)"', entrada)

    # Separar latitud y longitud si está en formato correcto
    lat, lon = "", ""
    if lat_lon:
        latlon_val = lat_lon.group(1)
        match = re.match(r"([\d\.\-]+ [NS]),\s*([\d\.\-]+ [EW])", latlon_val)
        if match:
            lat, lon = match.group(1), match.group(2)
        elif " " in latlon_val:  # formato como "52.5 N 4.0 W"
            partes = latlon_val.split()
            if len(partes) == 4:
                lat = f"{partes[0]} {partes[1]}"
                lon = f"{partes[2]} {partes[3]}"

    datos.append({
        "Nombre de muestra": nombre,
        "BioSample": biosample.group(1) if biosample else "",
        "Host": host.group(1) if host else "",
        "Accession": accession.group(1) if accession else "",
        "Geographic location": geo_loc.group(1) if geo_loc else "",
        "Latitude": lat,
        "Longitude": lon
    })

# Guardar como CSV
with open("honey-host.csv", "w", newline="", encoding="utf-8") as f:
    fieldnames = ["Nombre de muestra", "BioSample", "Host", "Accession", "Geographic location", "Latitude", "Longitude"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(datos)


In [3]:
import re
import csv

# Función para convertir coordenadas a decimal
def convertir_a_decimal(coord):
    if not coord:
        return ""
    match = re.match(r"([0-9.]+)\s*([NSWE])", coord)
    if not match:
        return ""
    valor = float(match.group(1))
    direccion = match.group(2)
    if direccion in ["S", "W"]:
        return -valor
    return valor

# Leer archivo
with open("honey-biosamples.txt", "r") as f:
    contenido = f.read()

# Separar muestras
entradas = re.split(r"\n\d+:\s", contenido)[1:]
nombres = re.findall(r"\n\d+:\s(.+)", contenido)

# Extraer información
datos = []

for nombre, entrada in zip(nombres, entradas):
    biosample = re.search(r"BioSample:\s*(SAMN\d+)", entrada)
    sra = re.search(r"SRA:\s*(SRS\d+)", entrada)
    host = re.search(r'/host="([^"]+)"', entrada)
    accession = re.search(r"Accession:\s*(SAMN\d+)", entrada)
    organism = re.search(r"Organism:\s*([^\n]+)", entrada)
    isolation = re.search(r'/isolation source="([^"]+)"', entrada)
    collection = re.search(r'/collection date="([^"]+)"', entrada)
    geo_loc = re.search(r'/geographic location="([^"]+)"', entrada)
    lat_lon = re.search(r'/latitude and longitude="([^"]+)"', entrada)

    # Inicializar latitud y longitud
    lat_decimal, lon_decimal = "", ""
    if lat_lon:
        val = lat_lon.group(1)
        partes = val.split()
        if len(partes) == 4:
            lat_decimal = convertir_a_decimal(f"{partes[0]} {partes[1]}")
            lon_decimal = convertir_a_decimal(f"{partes[2]} {partes[3]}")

    datos.append({
        "Nombre de muestra": nombre,
        "BioSample": biosample.group(1) if biosample else "",
        "SRA": sra.group(1) if sra else "",
        "Host": host.group(1) if host else "",
        "Accession": accession.group(1) if accession else "",
        "Organism": organism.group(1) if organism else "",
        "Isolation source": isolation.group(1) if isolation else "",
        "Collection date": collection.group(1) if collection else "",
        "Geographic location": geo_loc.group(1) if geo_loc else "",
        "Latitude": lat_decimal,
        "Longitude": lon_decimal
    })

# Guardar como CSV
with open("honey_metadata.csv", "w", newline="", encoding="utf-8") as f:
    fieldnames = [
        "Nombre de muestra", "BioSample", "SRA", "Host", "Accession",
        "Organism", "Isolation source", "Collection date",
        "Geographic location", "Latitude", "Longitude"
    ]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(datos)


In [4]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv("honey_metadata.csv")

# Mostrar los primeros registros
df.head()

,Nombre de muestra,BioSample,SRA,Host,Accession,Organism,Isolation source,Collection date,Geographic location,Latitude,Longitude
0,H1,SAMN03349284,SRS859919,Honey,SAMN03349284,honey metagenome,Honey,2012,"United Kingdom: Aberdovey, Gwynedd, Wales",52.587166,-4.083243
1,H1_2,SAMN03349285,SRS859928,Honey,SAMN03349285,honey metagenome,Honey,2012,"United Kingdom: Aberdovey, Gwynedd, Wales",52.587166,-4.083243
2,H2,SAMN03349286,SRS859920,Honey,SAMN03349286,honey metagenome,Honey,2012,"United Kingdom: East Cowes, Isle of Wight",50.735613,-1.269290
3,H3,SAMN03349287,SRS859921,Honey,SAMN03349287,honey metagenome,Honey,2012,"United Kingdom: Cwmpengraig, Carmarthenshire, ...",51.840725,-4.157574
4,H4,SAMN03349288,SRS859922,Honey,SAMN03349288,honey metagenome,Honey,2012,"United Kingdom: Dryslwyn, Carmarthenshire, Wales",51.876283,-4.105855


In [5]:
# Extraer año de 'Collection date'
df["Año"] = df["Collection date"].astype(str).str.extract(r"(\d{4})")

# Agrupar por Host y Lugar
def resumen_anios(grupo):
    anios = grupo["Año"].dropna().unique()
    anios_ordenados = sorted(anios)
    return pd.Series({
        "Número de muestras": len(grupo),
        "Años": ", ".join(anios_ordenados),
        "Año mínimo": min(anios_ordenados) if anios_ordenados else "",
        "Año máximo": max(anios_ordenados) if anios_ordenados else ""
    })

resumen = df.groupby(["Host", "Geographic location"]).apply(resumen_anios).reset_index()

# Mostrar
resumen

,Host,Geographic location,Número de muestras,Años,Año mínimo,Año máximo
0,Apis mellifera,Austria,34,2021,2021,2021
1,Apis mellifera,Benin,58,"2019, 2020, 2021",2019,2021
2,Apis mellifera,Brazil,8,"2020, 2021",2020,2021
3,Apis mellifera,Bulgaria,20,2019,2019,2019
4,Apis mellifera,Canada,10,"2020, 2021",2020,2021
...,...,...,...,...,...,...
65,not applicable,Greece: Evia,1,2018,2018,2018
66,not applicable,Greece: Mount Athos,2,2018,2018,2018
67,not applicable,Greece: Phocis,1,2018,2018,2018
68,not applicable,Sweden,38,2018,2018,2018


In [6]:
# Extraer el año de 'Collection date'
df["Año"] = df["Collection date"].astype(str).str.extract(r"(\d{4})")

# Agrupar por Host, Lugar y Año → contar muestras
resumen_por_año = (
    df.groupby(["Host", "Geographic location", "Año"])
    .size()
    .reset_index(name="Número de muestras")
)

# Mostrar
resumen_por_año

,Host,Geographic location,Año,Número de muestras
0,Apis mellifera,Austria,2021,34
1,Apis mellifera,Benin,2019,2
2,Apis mellifera,Benin,2020,52
3,Apis mellifera,Benin,2021,4
4,Apis mellifera,Brazil,2020,2
...,...,...,...,...
82,not applicable,Greece: Central Greece,2018,1
83,not applicable,Greece: Evia,2018,1
84,not applicable,Greece: Mount Athos,2018,2
85,not applicable,Greece: Phocis,2018,1


In [7]:
resumen_por_año.to_csv("resumen_host_lugar.csv", index=False)

In [8]:
import pandas as pd
import numpy as np

# Leer el CSV y preparar datos
df = pd.read_csv("honey_metadata.csv")
df["Año"] = df["Collection date"].astype(str).str.extract(r"(\d{4})")
df["Año"] = df["Año"].astype(float)  # Para poder ordenar y restar años

# Filtrar solo muestras de Apis mellifera
df_apis = df[df["Host"] == "Apis mellifera"]

# Lista para guardar muestras seleccionadas
muestras_seleccionadas = []

# Recorrer cada locación geográfica única
for locacion in df_apis["Geographic location"].unique():
    # Filtrar muestras de la locación actual
    muestras_loc = df_apis[df_apis["Geographic location"] == locacion]
    
    # Obtener años disponibles, ordenados descendente (más reciente primero)
    años_disponibles = sorted(muestras_loc["Año"].dropna().unique(), reverse=True)
    
    seleccionadas_loc = pd.DataFrame()
    
    # Intentar seleccionar 2 muestras empezando por el año más reciente
    for año in años_disponibles:
        # Filtrar muestras de ese año
        muestras_año = muestras_loc[muestras_loc["Año"] == año]
        
        # ¿Cuántas muestras hay disponibles?
        n_disponibles = len(muestras_año)
        
        # ¿Cuántas faltan para completar 2?
        n_faltan = 2 - len(seleccionadas_loc)
        
        if n_disponibles >= n_faltan:
            # Tomar solo las que faltan al azar
            seleccion = muestras_año.sample(n=n_faltan, random_state=42)
            seleccionadas_loc = pd.concat([seleccionadas_loc, seleccion])
            break  # Ya tenemos las 2, salimos del ciclo
        elif n_disponibles > 0:
            # Tomar todas las disponibles y seguir buscando en años anteriores
            seleccionadas_loc = pd.concat([seleccionadas_loc, muestras_año])
            # seguir buscando en años anteriores para completar 2 muestras
        # Si no hay muestras en ese año, continuar al siguiente año
    
    muestras_seleccionadas.append(seleccionadas_loc)

# Concatenar todas las muestras seleccionadas por locación
df_seleccionadas = pd.concat(muestras_seleccionadas)

print(df_seleccionadas)

# Opcional: guardar a CSV
df_seleccionadas.to_csv("seleccion_apis_por_locacion.csv", index=False)


                                      Nombre de muestra     BioSample  \
10      NBGW Honey H02 Apr 2015 (Early Season Foraging)  SAMN04478376   
11     NBGW Honey H04 Apr 2015  (Early Season Foraging)  SAMN04478377   
36    Metagenome or environmental sample from honey ...  SAMN10502527   
38    Metagenome or environmental sample from honey ...  SAMN10502529   
278   Metagenome or environmental sample from honey ...  SAMN13027025   
...                                                 ...           ...   
2108  Metagenome or environmental sample from honey ...  SAMN43381594   
1911  Metagenome or environmental sample from honey ...  SAMN43381397   
2146  Metagenome or environmental sample from honey ...  SAMN43381632   
1916  Metagenome or environmental sample from honey ...  SAMN43381402   
2155  Metagenome or environmental sample from honey ...  SAMN43381641   

              SRA            Host     Accession          Organism  \
10     SRS1278584  Apis mellifera  SAMN04478376  honey

In [9]:
# Hosts que quieres conservar todos
hosts_todos = [
    "Camponotus inflatus", 
    "Melipona beecheii", 
    "Melipona bicolor", 
    "Melipona mondury", 
    "Melipona quadrifasciata",
    "Tetragonula pagdeni"
]

# Hosts de los que quieres tomar sólo 6 muestras al azar
hosts_submuestreo = ["hybrid"]

# Filtrar todos los hosts que quieres conservar completos
df_todos = df[df["Host"].isin(hosts_todos)]

# Filtrar los hosts para submuestreo
df_submuestra = df[df["Host"].isin(hosts_submuestreo)]

# Seleccionar 6 muestras al azar por cada host en hosts_submuestreo (o menos si no hay suficientes)
df_submuestra_seleccionada = (
    df_submuestra.groupby("Host")
    .apply(lambda x: x.sample(n=min(len(x), 6), random_state=42))
    .reset_index(drop=True)
)

# Concatenar todos juntos
df_final = pd.concat([df_todos, df_submuestra_seleccionada], ignore_index=True)

# Opcional: guardar resultado
df_final.to_csv("muestras_filtradas_no_APIS.csv", index=False)

print(df_final)


                                    Nombre de muestra     BioSample  \
0   Metagenome or environmental sample from honey ...  SAMN17080697   
1   Metagenome or environmental sample from honey ...  SAMN17080698   
2   Metagenome or environmental sample from honey ...  SAMN17080699   
3   Metagenome or environmental sample from honey ...  SAMN17080700   
4   Metagenome or environmental sample from honey ...  SAMN17080701   
5                    Melipona honey sample procariota  SAMN27112593   
6                        Melipona honey sample fungal  SAMN27113155   
7                            Honeypot Ant Honey Plate  SAMN34246860   
8                            Honeypot Ant Honey Plate  SAMN34246865   
9                                      16S_Honey_Mbb1  SAMN41405254   
10                                     16S_Honey_Mbb2  SAMN41405255   
11                                     16S_Honey_Mbb3  SAMN41405256   
12                                      16S_Honey_Mm1  SAMN41405257   
13    

In [10]:
import re
import pandas as pd

# Leer todo el archivo
with open("honey-biosamples.txt", "r", encoding="utf-8") as f:
    texto = f.read()

# Dividir por muestras según patrón: número seguido de ":"
bloques = re.split(r'\n\d+:\s', "\n" + texto)  
# El primer elemento puede estar vacío porque pusimos "\n" al inicio
bloques = [b for b in bloques if b.strip() != ""]

datos = []

for bloque in bloques:
    # Extraer BioSample
    bio_match = re.search(r'BioSample:\s*([^;]+)', bloque)
    biosample = bio_match.group(1).strip() if bio_match else None
    
    # Extraer SRA
    sra_match = re.search(r'SRA:\s*([^\n;]+)', bloque)
    sra = sra_match.group(1).strip() if sra_match else None
    
    # Extraer Host
    host_match = re.search(r'/host="([^"]+)"', bloque)
    host = host_match.group(1).strip() if host_match else None
    
    # Otros campos: Accession, Geographic location, Latitude y Longitude, Organism, Isolation source, Collection date
    accession_match = re.search(r'Accession:\s*([^\s]+)', bloque)
    accession = accession_match.group(1).strip() if accession_match else None
    
    geo_match = re.search(r'/geographic location="([^"]+)"', bloque)
    geographic_location = geo_match.group(1).strip() if geo_match else None
    
    latlong_match = re.search(r'/latitude and longitude="([^"]+)"', bloque)
    latlong = latlong_match.group(1).strip() if latlong_match else None
    lat = None
    lon = None
    if latlong and latlong.lower() != "not applicable":
        # Intentamos dividir por espacio, si está en formato "lat N lon W"
        parts = latlong.split()
        if len(parts) >= 2:
            lat = parts[0]
            lon = parts[1]
    
    organism_match = re.search(r'Organism:\s*([^\n]+)', bloque)
    organism = organism_match.group(1).strip() if organism_match else None
    
    iso_match = re.search(r'/isolation source="([^"]+)"', bloque)
    isolation_source = iso_match.group(1).strip() if iso_match else None
    
    date_match = re.search(r'/collection date="([^"]+)"', bloque)
    collection_date = date_match.group(1).strip() if date_match else None

    # Guardar en lista
    datos.append({
        "BioSample": biosample,
        "SRA": sra,
        "Host": host,
        "Accession": accession,
        "Geographic location": geographic_location,
        "Latitude": lat,
        "Longitude": lon,
        "Organism": organism,
        "Isolation source": isolation_source,
        "Collection date": collection_date
    })

df = pd.DataFrame(datos)

# Revisar si hay NaN en BioSample o SRA para 'hybrid'
print(df[df["Host"] == "hybrid"][["BioSample", "SRA"]])

# Guardar
df.to_csv("salidad.csv", index=False)


        BioSample        SRA
818  SAMD00220018  DRS159442
819  SAMD00220019  DRS159443
820  SAMD00220020  DRS159444
821  SAMD00220021  DRS159445
822  SAMD00220022  DRS159446
823  SAMD00220023  DRS159447
824  SAMD00220024  DRS159448
825  SAMD00220025  DRS159449
826  SAMD00220026  DRS159450
827  SAMD00220027  DRS159451
828  SAMD00220028  DRS159452
829  SAMD00220029  DRS159453
830  SAMD00220030  DRS159454
831  SAMD00220031  DRS159455
832  SAMD00220032  DRS159456
833  SAMD00220033  DRS159457
834  SAMD00220034  DRS159458
835  SAMD00220035  DRS159459
836  SAMD00220036  DRS159460
837  SAMD00220037  DRS159461
838  SAMD00220038  DRS159462
839  SAMD00220039  DRS159463
840  SAMD00220040  DRS159464
841  SAMD00220041  DRS159465
842  SAMD00220042  DRS159466
843  SAMD00220043  DRS159467
844  SAMD00220044  DRS159468
845  SAMD00220045  DRS159469
846  SAMD00220046  DRS159470
847  SAMD00220047  DRS159471
848  SAMD00220048  DRS159472
849  SAMD00220049  DRS159473
